# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [192]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [193]:
import json
import numpy as np

## Settings / Constants

In [194]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment"]#, "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [195]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}", "Llama70B": "\\textbf{Llama-2-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

N_METRICS = {"aspect_category": 6, "aspect_category_sentiment": 6,
             "end_2_end_absa": 6, "target_aspect_sentiment_detection": 5}

## Code

In [196]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [197]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n\n", "-------#-----#-----#-------")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)
        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")
        else:
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {add_thousand_dots(round_number(results['eval_f1'], 3))} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(["fixed", "random"]):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1_micro'], 3))} & {add_thousand_dots(round_number(results['eval_f1_macro'], 3))}"
                else:
                    f1_metrics = f"{add_thousand_dots(round_number(results['eval_f1'], 3))}"
                print(
                    f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} & {add_thousand_dots(round_number(results['eval_accuracy'], 3))} \\\\")

                # print(llm_idx, fs_idx, freq_idx)
            if freq_idx == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{2-"+str(N_METRICS[absa_task]+1)+"}\\arrayrulecolor{black}")
            else:
                print("\\hline")
        print("\hline")
    print("\n\n")

aspect_category :

 -------#-----#-----#-------
\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.910 & 0.899 & 0.841 \\
 &  & 1,000 & 0 & 0.916 & 0.906 & 0.852 \\
 &  & 2,000 & 0 & 0.926 & 0.916 & 0.868 \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 0.768 & 0.758 & 0.630 \\
 &  & 25 & 975 & 0.764 & 0.755 & 0.626 \\
 &  & 25 & 1,975 & 0.779 & 0.764 & 0.656 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.885 & 0.876 & 0.800 \\
 &  & 500 & 1,000 & 0.873 & 0.863 & 0.782 \\
 &  & 500 & 1,500 & 0.870 & 0.862 & 0.780 \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
\hline
\multirow{6}{*}{\textbf{GPT-3.5-turbo}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 0.807 & 0.804 & 0.682 \\
 &  & 25 & 975 & 0.793 & 0.788 & 0.672 \\
 &  & 25 & 1,975 & 0.794 & 0.789 & 0.670 \\
\arrayrulecolor{gray}\cline{2-7}\arra

### Report Metrics Fine-Grained Metrics

#### ACD

In [198]:
absa_task = "aspect_category"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
        for metric in ["f1", "accuracy"]:
            class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
    for llm_idx, llm in enumerate(LLMS):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)
            # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
            if absa_task == "TASD":
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])} & {add_thousand_dots(results['eval_f1_macro'])}"
            else:
                f1_metrics = f"{add_thousand_dots(results['eval_f1_micro'])}"

            if llm_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                    class_wise_metrics += f"{add_thousand_dots(round_number(results[f'eval_{metric}_{ac}'], 3))} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if freq_idx == 2 and llm_idx == 0:
               print(
                "\\arrayrulecolor{gray}\cline{2-14}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.858 &0.932 &0.939 &0.948 &0.928 &0.954 &0.863 &0.962 &0.909 &0.986  \\
 &  & 1,000 & 0 & 0.860 &0.932 &0.944 &0.952 &0.942 &0.963 &0.865 &0.962 &0.919 &0.987  \\
 &  & 2,000 & 0 & 0.881 &0.941 &0.954 &0.961 &0.946 &0.965 &0.878 &0.965 &0.921 &0.988  \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.822 &0.909 &0.915 &0.929 &0.912 &0.945 &0.851 &0.959 &0.880 &0.981  \\
 &  & 500 & 1,000 & 0.818 &0.905 &0.901 &0.919 &0.904 &0.941 &0.826 &0.952 &0.864 &0.979  \\
 &  & 500 & 1,500 & 0.811 &0.901 &0.895 &0.915 &0.900 &0.938 &0.840 &0.956 &0.865 &0.979  \\
\arrayrulecolor{gray}\cline{2-14}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.864 &0.931 &0.933 &0.943 &0.924 &0.953 &0.862 &0.963 &0.915 &0.987  \\
 &  & 500 & 1,000 & 0.858 &0.928 &0.933 &0.943 &0.920 &0.950 &0.867 &0.964 &0.907 &0.985  \\
 &  & 500 & 1,5

#### ACSA

In [199]:
absa_task = "aspect_category_sentiment"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for llm_idx, llm in enumerate(LLMS):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if llm_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {add_thousand_dots(round_number(results[f'{metric}_{ac}-{polarity}'], 3))} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 10 if ac_idx == 2 else 16
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.828 & 0.956 & 0.159 & 0.992 & 0.825 & 0.964 & 0.924 & 0.963 & 0.763 & 0.983 & 0.826 & 0.943 \\
 &  & 1,000 & 0 & 0.852 & 0.962 & 0.667 & 0.995 & 0.838 & 0.966 & 0.933 & 0.968 & 0.839 & 0.988 & 0.855 & 0.952 \\
 &  & 2,000 & 0 & 0.850 & 0.961 & 0.634 & 0.994 & 0.848 & 0.968 & 0.942 & 0.972 & 0.832 & 0.987 & 0.859 & 0.954 \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.789 & 0.945 & 0.317 & 0.989 & 0.819 & 0.961 & 0.886 & 0.947 & 0.598 & 0.972 & 0.812 & 0.939 \\
 &  & 500 & 1,000 & 0.644 & 0.932 & 0.308 & 0.987 & 0.669 & 0.945 & 0.715 & 0.911 & 0.450 & 0.969 & 0.674 & 0.924 \\
 &  & 500 & 1,500 & 0.511 & 0.918 & 0.251 & 0.990 & 0.530 & 0.937 & 0.579 & 0.879 & 0.344 & 0.965 & 0.530 & 0.906 \\
\arrayrulecolor{gray}\cline{2-16}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 

In [200]:
llllllllllllllllllllll

NameError: name 'llllllllllllllllllllll' is not defined